In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Get the project root directory
# Try to find sciborg directory by going up from current directory
current_dir = Path.cwd()
project_root = current_dir

# Look for sciborg in the path or go up until we find it
while 'sciborg' not in str(project_root.name) and project_root != project_root.parent:
    project_root = project_root.parent
    if project_root == project_root.parent:  # Reached filesystem root
        # Fallback: assume we're in tests/notebooks, go up two levels
        project_root = Path.cwd()
        if 'notebooks' in str(project_root):
            project_root = project_root.parent.parent
        break

# If still not found, use current directory
if 'sciborg' not in str(project_root):
    project_root = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd()
    if 'notebooks' in str(project_root):
        project_root = project_root.parent.parent

# Load .env file from project root
env_path = project_root / '.env'
if env_path.exists():
    load_dotenv(dotenv_path=str(env_path))
else:
    # Try loading from current directory
    load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")

if openai_key:
    os.environ["OPENAI_API_KEY"] = openai_key

In [ ]:
import os
import sys
from pathlib import Path

# Get the project root directory (reuse from previous cell if available, otherwise recalculate)
if 'project_root' not in globals():
    current_dir = Path.cwd()
    project_root = current_dir
    # Look for sciborg in the path
    while 'sciborg' not in str(project_root.name) and project_root != project_root.parent:
        project_root = project_root.parent
        if project_root == project_root.parent:
            project_root = Path.cwd()
            if 'notebooks' in str(project_root):
                project_root = project_root.parent.parent
            break

sys.path.insert(0, str(project_root))

from langchain_openai import ChatOpenAI
from sciborg.ai.agents.core import create_sciborg_chat_agent
from sciborg.ai.chains.microservice import module_to_microservice
from sciborg.ai.chains.workflow import create_workflow_planner_chain, create_workflow_constructor_chain

from sciborg.utils.drivers import MicrowaveSynthesizer, MicrowaveSynthesizerObject, PubChemCaller
from sciborg.core.library.base import BaseDriverMicroservice

/Users/atharvaparikh/miniconda3/envs/sciborgdev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### PubChem Microservice

In [ ]:
# Use relative path from project root
file_path = project_root / 'ai' / 'agents' / 'driver_pubchem.json'

pubchem_command_microservice = module_to_microservice(PubChemCaller)
print(pubchem_command_microservice)

with open(file_path, 'w') as outfile:
    outfile.write(pubchem_command_microservice.model_dump_json(indent=2))

Processing functions:   0%|          | 0/8 [00:00<?, ?function/s]

Processing functions:  12%|█▎        | 1/8 [00:03<00:27,  3.87s/function]

{'name': 'get_assay_description_from_AID', 'microservice': 'PubChemCaller', 'desc': 'Get high level Assay description, protocol and comment on the scores for a given assay id', 'uuid': '508d2531-6475-4e9b-98d6-baf8c6a0289d', 'parameters': {'aid': {'name': 'aid', 'data_type': 'str', 'desc': 'string representation of a list of assay IDs'}}, 'has_return': True, 'return_signature': {'dict': 'a dictionary of description, protocol and comment for the given assay ID'}}
<class 'dict'>


Processing functions:  25%|██▌       | 2/8 [00:07<00:23,  3.91s/function]

{'name': 'get_assay_name_from_aid', 'microservice': 'PubChemCaller', 'uuid': '508d2531-6475-4e9b-98d6-baf8c6a0289d', 'desc': 'Gives a dictionary of names for each assay ID (aid)', 'parameters': {'aid': {'name': 'aid', 'data_type': 'str', 'desc': 'string representation of a list of assay IDs'}}, 'has_return': True, 'return_signature': {'str': 'a dictionary of names for each assay ID'}}
<class 'dict'>


Processing functions:  38%|███▊      | 3/8 [00:20<00:40,  8.13s/function]

{'name': 'get_assay_results_for_compund', 'microservice': 'PubChemCaller', 'desc': 'Gets all the assay results for a provided compound with an optional filter of assay activity type.', 'uuid': '508d2531-6475-4e9b-98d6-baf8c6a0289d', 'parameters': {'cid': {'name': 'cid', 'data_type': 'str', 'desc': 'a single cid representing a compound', 'is_optional': False}, 'activity_name': {'name': 'activity_name', 'data_type': 'str', 'desc': 'the specific activity type to filter on for example Ki, Kd, IC50, etc', 'is_optional': True}}, 'has_return': True, 'return_signature': {'dict': 'a disctionary representation of the assay result table'}}
<class 'dict'>


Processing functions:  50%|█████     | 4/8 [00:27<00:29,  7.43s/function]

{'name': 'get_cid_from_name', 'microservice': 'PubChemCaller', 'uuid': '508d2531-6475-4e9b-98d6-baf8c6a0289d', 'desc': 'Get the cid from chemical name', 'parameters': {'inp': {'name': 'inp', 'data_type': 'str', 'desc': 'string representation of a chemical name'}}, 'has_return': True, 'return_signature': {'dict': 'a dictionary of cids for the given chemical name'}}
<class 'dict'>


In [ ]:
pubchem_driver_based_agent = create_sciborg_chat_agent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    verbose=True
)

### Microwave Synthsizer Microservice

In [ ]:
# Use relative path from project root
file_path = project_root / 'ai' / 'agents' / 'driver_MicroSynth.json'

driver_command_microservice = module_to_microservice(MicrowaveSynthesizer)
print(type(driver_command_microservice))

with open(file_path, 'w') as outfile:
    outfile.write(driver_command_microservice.model_dump_json(indent=2))

{'name': 'allocate_session', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Allocates a session on the microwave synthesizer. Must be called prior to any other action.', 'uuid': '01458297-1d8a-4956-a063-4f25558a771e', 'parameters': {}, 'has_return': True, 'return_signature': {'session_ID': 'the id of the allocated session'}}
<class 'dict'>
{'name': 'close_lid', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Closes the lid on the microwave synthesizer. Must be run prior to running heating.', 'uuid': '01458297-1d8a-4956-a063-4f25558a771e', 'parameters': {'session_ID': {'name': 'session_ID', 'data_type': 'str', 'is_optional': False, 'is_list': False, 'desc': 'the id of the current session'}}, 'has_return': True, 'return_signature': {'status': 'is a status string that provides the result of the operation'}}
<class 'dict'>
{'name': 'get_precent_conversion', 'microservice': 'MicrowaveSynthesizer', 'desc': 'Gets the precent conversion of synthesis after running the experiment. Can only be

ValidationError: 4 validation errors for BaseInfoCommand
parameters.vial_num.precision.literal[-1]
  Input should be -1 [type=literal_error, input_value={'const': -1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error
parameters.vial_num.precision.constrained-int
  Input should be a valid integer [type=int_type, input_value={'const': -1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/int_type
parameters.session_ID.precision.literal[-1]
  Input should be -1 [type=literal_error, input_value={'const': -1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error
parameters.session_ID.precision.constrained-int
  Input should be a valid integer [type=int_type, input_value={'const': -1}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/int_type

# PubChem Agent

In [ ]:
test_agent = create_sciborg_chat_agent(
    microservice=pubchem_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    verbose=True
)

test_agent.invoke(
    {
        "input": "What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?"
    }
)



> Entering new AgentExecutor chain...
Thought: I need to first get the CID of pzm21 and then retrieve the assay results for this compound. I'll begin with getting the CID.

Action:
```
{
    "action": "get_cid_from_name",
    "action_input": {
        "inp": "pzm21"
    }
}
```{'response': {'121596705', '', '124186394'}}The CID for pzm21 is obtained and has multiple entries: 121596705, 124186394. Now I will use one of the CIDs - 121596705, to get the assay results for this compound, specifically looking for results related to 'Ki' activity.

Action:
```
{
    "action": "get_assay_results_for_compund",
    "action_input": {
        "cid": "121596705",
        "activity_name": "Ki"
    }
}
```{'AID': {1: '1573390', 5: '1609954', 17: '1806958', 18: '1806958', 19: '1806958', 20: '1806958'}, 'Panel Member ID': {1: '', 5: '', 17: '', 18: '', 19: '', 20: ''}, 'SID': {1: '440143506', 5: '440143506', 17: '440726618', 18: '440726618', 19: '440726618', 20: '440726618'}, 'CID': {1: '121596705', 

{'input': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
 'chat_history': '',
 'past_action_log': '',
 'output': "The 'Ki' value of pzm21 to the Mu opioid receptor was found to be 0.0011 uM in an assay with ID '1573390'. The assay was part of a study titled 'Biased Ligands of G Protein-Coupled Receptors (GPCRs): Structure-Functional Selectivity Relationships (SFSRs) and Therapeutic Potential.' The assay tested the compound's activity in the displacement of [3H]-Diprenorphine from the mu opioid receptor expressed in sf9 insect cell membranes. The result was flagged as active in the PubChem assay presentation as it had activity <= 10uM. [Source: PubChem]",
 'intermediate_steps': [(AgentAction(tool='get_cid_from_name', tool_input={'inp': 'pzm21'}, log='Thought: I need to first get the CID of pzm21 and then retrieve the assay results for this compound. I\'ll begin with getting the CID.\n\nAction:\n```\n{\n    "action": "get_cid_from_name",\n  

How parameters change:

- ❌ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction

In [6]:
#lets try using a planner chain here
planner = create_workflow_planner_chain(
    llm=ChatOpenAI(model='gpt-4'),
    library=pubchem_command_microservice
)

In [7]:
planner.invoke(
    {
        "query": "What is the IC50 of 1-[(2S)-2-(dimethylamino)-3-(4-hydroxyphenyl)propyl]-3-[(2S)-1-thiophen-3-ylpropan-2-yl]urea to the Mu opioid receptor, cite a specific assay in your response?"
    }
)

{'query': 'What is the IC50 of 1-[(2S)-2-(dimethylamino)-3-(4-hydroxyphenyl)propyl]-3-[(2S)-1-thiophen-3-ylpropan-2-yl]urea to the Mu opioid receptor, cite a specific assay in your response?',
 'text': '1. Use the `get_cid_from_name` function to retrieve the CID for the chemical name "1-[(2S)-2-(dimethylamino)-3-(4-hydroxyphenyl)propyl]-3-[(2S)-1-thiophen-3-ylpropan-2-yl]urea".\n   - Parameter:\n     - `inp`: "1-[(2S)-2-(dimethylamino)-3-(4-hydroxyphenyl)propyl]-3-[(2S)-1-thiophen-3-ylpropan-2-yl]urea"\n   - Save the returned CID as `compound_cid`.\n\n2. Use the `get_assay_results_for_compund` function to get the assay results for the retrieved CID, and specifically filter for the IC50 activity.\n   - Parameters:\n     - `cid`: `compound_cid`\n     - `activity_name`: "IC50"\n   - Save the returned assay result table as `assay_results`.\n\n3. From the `assay_results`, find the assay result related to the Mu opioid receptor and save the assay ID as `assay_id`.\n\n4. Use the `get_assay_na

In [ ]:
Pubchem_agent = create_sciborg_chat_agent(
    microservice=driver_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    agent_as_a_tool=pubchem_driver_based_agent,
    verbose=True,
    **{"trim_intermediate_steps" : 3}
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': ''} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n{additional_instructions}\nYou may not need to use a tool to answer certian questions. You can instead refer to your domain of knowledge or the chat history and then return your "Final Answer".\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n    "a

In [9]:
Pubchem_agent.invoke(
    {
        "input": "What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?"
    }
)

#NOTE:
# Content length exceeded in the response error. All the ids that are fetched as response are flooding the response and filling up the content length. 
# Also another error is that it asked from human input when the flag as set to false


#Answer along the lines of 0.000004 µM



> Entering new AgentExecutor chain...
Action:
```
{
    "action": "call_provided_Agent",
    "action_input": {
        "question": "What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?"
    }
}
```

> Entering new AgentExecutor chain...
Thought: The user is asking for the Ki (inhibition constant) of the compound pzm21 to the Mu opioid receptor. The Ki value can be found in assay results. But first, I need to get the compound ID (CID) for pzm21 from its name. 

Action:
```
{
    "action": "get_cid_from_name",
    "action_input": {
        "inp": "pzm21"
    }
}
```{'response': {'', '121596705', '124186394'}}The response indicates that there are multiple compound IDs associated with the name "pzm21". I need to use the obtained CIDs, specifically the CID '121596705' to fetch the assay results. The assay results will contain the Ki value for the compound's interaction with the Mu opioid receptor.

Action:
```
{
    "action": "get_assay_results_for_co

{'input': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?',
 'chat_history': '',
 'past_action_log': '',
 'output': "The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM as determined by the assay with AID '1573390'. The assay is titled 'Biased Ligands of G Protein-Coupled Receptors (GPCRs): Structure-Functional Selectivity Relationships (SFSRs) and Therapeutic Potential.' The related publication can be found in the Journal of Medicinal Chemistry, volume 61, issue 22, pages 9841-9878, in the year 2018 (DOI: 10.1021/acs.jmedchem.8b00435). The assay targets the Mu opioid receptor (ChEMBL ID: CHEMBL233). Please note that compounds with activity <= 10uM or explicitly reported as active by ChEMBL are flagged as active in this PubChem assay presentation. [Data sourced from PubChem]",
 'intermediate_steps': [(AgentAction(tool='call_provided_Agent', tool_input={'question': 'What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in

In [10]:
print(Pubchem_agent.memory.memories[0].buffer)

Human: What is the Ki of pzm21 to the Mu opioid receptor, cite a specific assay in your response?
AI: The Ki of pzm21 to the Mu opioid receptor is 0.0011 uM as determined by the assay with AID '1573390'. The assay is titled 'Biased Ligands of G Protein-Coupled Receptors (GPCRs): Structure-Functional Selectivity Relationships (SFSRs) and Therapeutic Potential.' The related publication can be found in the Journal of Medicinal Chemistry, volume 61, issue 22, pages 9841-9878, in the year 2018 (DOI: 10.1021/acs.jmedchem.8b00435). The assay targets the Mu opioid receptor (ChEMBL ID: CHEMBL233). Please note that compounds with activity <= 10uM or explicitly reported as active by ChEMBL are flagged as active in this PubChem assay presentation. [Data sourced from PubChem]


In [11]:
print(Pubchem_agent.memory.memories[1].buffer)

The human inquires about the Ki of pzm21 to the Mu opioid receptor and requests a specific assay to be cited in the response.


# RAG on Initiator+ Docs

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ❌ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction
- ❌ microservice : object - Microservice object

In [ ]:
rag_agent = create_sciborg_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = str(project_root / 'embeddings' / 'NIH_docs_embeddings'),
    use_pubchem=False,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': '\nYou main responsibility is to provide the information that is requested by the human. You have access to the relevant information that is stored in the documents. You can use the call_RAG_chain tool to query the information from the documents. To use this tool you need to frame a question that is clear and concise and then use the call_RAG_chain tool to get the information from the documents. \n\nIf the user has specified the source then use information retrieved from that source to provide the answer.\n\nIf the information is not available in the documents, you can respond to the human by giving a general response from your domain of knowledge but remember to specify that this information is not from the documents and hen

In [19]:
rag_agent.invoke(
    {
        "input": "How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?"
    }
)



> Entering new AgentExecutor chain...


Thought: This is a scientific question about the influence of microwave irradiation on reaction mechanisms compared to conventional heating methods. As an AI, I don't have the knowledge to answer this directly. I will use the call_RAG_agent tool to search for the answer in the relevant scientific literature.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "What is the difference between microwave irradiation and conventional heating methods in influencing reaction mechanisms?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'difference between microwave irradiation and conventional heating methods in influencing reaction mechanisms'}`


[Document(page_content='Under the heating activity, the dividers of the reaction vessel act as\nthe mediator and collect the heat from the exterior heating source,\nexchanging the thermal energy to the reaction solvent. Then, re-\nactants start to tr

{'input': 'How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'Microwave irradiation provides a more uniform heating to the reaction materials in the entire reaction vessel, which allows for rapid and uniform heat transfer. This is different from conventional heating methods where thermal gradients can lead to non-uniform transformations and issues in scale-up production. Moreover, microwave radiation collaborates with the reaction material and generates heat simultaneously, leading to faster reaction times compared to conventional heating methods. For instance, the reaction time for compound synthesis is significantly shorter when using microwave irradiation compared to conventional heating methods.',
 'intermediate_steps': [(AgentAction(tool='call_RAG_agent', tool_input={'question': 'What is the difference between microwave irradiation and conventi

In [20]:
rag_agent.invoke(
    {
        "input": "What function does the pressure setting serve on the Biotage Initiator+?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking about the function of the pressure setting on the Biotage Initiator+, a microwave synthesizer. This question seems to be about the role of pressure in the reaction process in the context of microwave synthesis. I don't have this specific information, so I'll use the call_RAG_agent tool to search for the answer.

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {
        "question": "What is the function of the pressure setting on a microwave synthesizer like the Biotage Initiator+?"
    }
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'function of the pressure setting on a microwave synthesizer like the Biotage Initiator+'}`


[Document(page_content='features offer faster and more powerful heating (400 W) of \na broader range of solvents. The setting for low microwave \nabsorbing solvents enhances the heating for e.g. toluene and \n1, 4-dioxan

{'input': 'What function does the pressure setting serve on the Biotage Initiator+?',
 'chat_history': 'Human: How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?\nAI: Microwave irradiation provides a more uniform heating to the reaction materials in the entire reaction vessel, which allows for rapid and uniform heat transfer. This is different from conventional heating methods where thermal gradients can lead to non-uniform transformations and issues in scale-up production. Moreover, microwave radiation collaborates with the reaction material and generates heat simultaneously, leading to faster reaction times compared to conventional heating methods. For instance, the reaction time for compound synthesis is significantly shorter when using microwave irradiation compared to conventional heating methods.',
 'past_action_log': 'The human inquires about the influence of microwave irradiation on reaction mechanisms compared to 

In [21]:
rag_agent.invoke(
    {
        "input": "What Conditions Are Appropriate When Performing Microwave Synthesis?"
    }
)



> Entering new AgentExecutor chain...
Thought: The human is asking for the appropriate conditions when performing microwave synthesis. This involves setting the right parameters for the duration, temperature, and pressure of the microwave synthesizer. We can specify general guidelines for these parameters based on the function documentation for the `update_heating_parameters` tool.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The appropriate conditions for performing microwave synthesis with a microwave synthesizer typically involve setting the right parameters for the duration, temperature, and pressure. Here are some general guidelines:\n\n1. Duration: The duration is usually set between 1 and 60 minutes, depending on the particular reaction you're running.\n\n2. Temperature: The temperature can be set anywhere between 25 and 250 degrees Celsius. The exact temperature will depend on the specific reaction and the materials being used.\n\n3. Pressure: The pressure sh

{'input': 'What Conditions Are Appropriate When Performing Microwave Synthesis?',
 'chat_history': 'Human: How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?\nAI: Microwave irradiation provides a more uniform heating to the reaction materials in the entire reaction vessel, which allows for rapid and uniform heat transfer. This is different from conventional heating methods where thermal gradients can lead to non-uniform transformations and issues in scale-up production. Moreover, microwave radiation collaborates with the reaction material and generates heat simultaneously, leading to faster reaction times compared to conventional heating methods. For instance, the reaction time for compound synthesis is significantly shorter when using microwave irradiation compared to conventional heating methods.\nHuman: What function does the pressure setting serve on the Biotage Initiator+?\nAI: The pressure setting on a microwave synt

In [22]:
print(rag_agent.memory.memories[0].buffer)

Human: How does microwave irradiation influence reaction mechanisms differently compared to conventional heating methods?
AI: Microwave irradiation provides a more uniform heating to the reaction materials in the entire reaction vessel, which allows for rapid and uniform heat transfer. This is different from conventional heating methods where thermal gradients can lead to non-uniform transformations and issues in scale-up production. Moreover, microwave radiation collaborates with the reaction material and generates heat simultaneously, leading to faster reaction times compared to conventional heating methods. For instance, the reaction time for compound synthesis is significantly shorter when using microwave irradiation compared to conventional heating methods.
Human: What function does the pressure setting serve on the Biotage Initiator+?
AI: The pressure setting on a microwave synthesizer like the Biotage Initiator+ facilitates reactions to occur at higher temperatures and pressures

In [43]:
print(rag_agent.memory.memories[1].buffer)

The human inquires about the influence of microwave irradiation on reaction mechanisms compared to conventional heating methods. The AI has not provided a response yet to the question. The human then asks about the function of the pressure setting on the Biotage Initiator+. The human now asks about the appropriate conditions for performing microwave synthesis.


In [24]:
print(rag_agent.memory.memories[2].buffer)

The AI agent initially used the call_RAG_agent tool to search for information on how microwave irradiation influences reaction mechanisms compared to conventional heating methods. It found that microwave irradiation provides uniform heating to reaction materials in the entire vessel, enabling rapid and uniform heat transfer, in contrast to conventional heating methods. The AI also found that microwave radiation collaborates with the reaction material and generates heat simultaneously, leading to faster reaction times. The AI referred to scientific literature to answer this question.

In a new conversation, the AI used the call_RAG_agent tool to search for information on the function of the pressure setting on a microwave synthesizer like the Biotage Initiator+. It found that the pressure setting allows for higher temperatures and pressures of up to 300 °C and 30 bar, enabling the performance of difficult reactions and allowing solvents with low boiling points to be run at higher temper

# Initiator+ and RAG on Initiator+ Docs

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ❌ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ✅ human_interaction: `bool` - Whether to use human interaction

In [ ]:
initiator_and_rag_agent = create_sciborg_chat_agent(
    microservice=driver_command_microservice,
    rag_vectordb_path = str(project_root / 'embeddings' / 'NIH_Aco_only_doc_embeddings'),
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    agent_as_a_tool=False,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': "\nIf you are unsure about a specific request or any information you observe use the 'human' tool to ask the human for guidance.\n\nDon't just use default values for tool inputs, look at the chat history to see if any values were provided previosuly and if not ask the human for input and provide the human with the default values if requested.\n"} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_in

Ask the agent to load a specific vial which has reactants for a n-alkylation reaction. Ask the agent to suggest heating parameters for this reaction and confirm with you prior to setting the parameters.

In [28]:
initiator_and_rag_agent.invoke(
    {
        "input": "Can you heat a vial for me?"
    }
)

#further question would be: recommend paramaters for n-alkylation reaction using 3-Bromoquinoline and morpholine [II] as reactants



> Entering new AgentExecutor chain...
Question: Can you heat a vial for me?

Thought: Before heating a vial, we need a session ID, a vial number to load into the microwave synthesizer, the lid must be closed, and heating parameters need to be set. So, first, let's allocate a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': '8e60e389-11ae-4dbb-b3f8-e819b9c26746'}Now that we have a session ID, we need to load a vial into the microwave synthesizer. But before we load the vial, we need to know which vial number to load. Since the user did not specify a vial number, I need to ask for it.

Action:
```
{
    "action": "human",
    "action_input": {
        "query": "Which vial number would you like to load into the microwave synthesizer? Please provide a number between 1 and 10."
    }
}
```

Which vial number would you like to load into the microwave synthesizer? Please provide a number between 1 and 10.
5Now that we have the vial number,

{'input': 'Can you heat a vial for me?',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'The vial is now heating with the provided parameters.',
 'intermediate_steps': [(AgentAction(tool='allocate_session', tool_input={}, log='Question: Can you heat a vial for me?\n\nThought: Before heating a vial, we need a session ID, a vial number to load into the microwave synthesizer, the lid must be closed, and heating parameters need to be set. So, first, let\'s allocate a session.\n\nAction:\n```\n{\n    "action": "allocate_session",\n    "action_input": {}\n}\n```'),
   "{'session_ID': '8e60e389-11ae-4dbb-b3f8-e819b9c26746'}"),
  (AgentAction(tool='human', tool_input={'query': 'Which vial number would you like to load into the microwave synthesizer? Please provide a number between 1 and 10.'}, log='Now that we have a session ID, we need to load a vial into the microwave synthesizer. But before we load the vial, we need to know which vial number to load. Since the user 

In [29]:
print(initiator_and_rag_agent.memory.memories[0].buffer)

Human: Can you heat a vial for me?
AI: The vial is now heating with the provided parameters.


In [30]:
print(initiator_and_rag_agent.memory.memories[1].buffer)

The human asks the AI to heat a vial. The AI explains the steps required to heat the vial, including allocating a session, opening the lid, loading the vial, closing the lid, setting heating parameters, and finally heating the vial.


In [31]:
print(initiator_and_rag_agent.memory.memories[2].buffer)

The AI was asked to heat a vial and had to ask the user for the vial number to load into the microwave synthesizer. After receiving the vial number, the AI successfully closed the lid and asked the user for the heating parameters including duration, temperature, and pressure. The user did not provide these parameters, so the AI had to ask for them. The AI then used the call_RAG_agent tool to find the recommended heating parameters for an n-alkylation reaction using specific reactants. The recommended parameters were a pressure of 1 bar and a temperature of 120°C. The AI then asked the user for the duration of the heating process, which was provided as 50 minutes. The AI referred to the following list of documents for this information: n-alkylation_reactions.pdf (page 5), recommended_heating_parameters.pdf (page 10), and reaction_procedures.pdf (page 15). Around 10 documents were referred to get this information.


# Initiator+ and PubChem Agent

How parameters change:

- ❌ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ✅ human_interaction: `bool` - Whether to use human interaction

In [ ]:
initiator_and_pubchem_agent = create_sciborg_chat_agent(
    microservice=driver_command_microservice,
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=True,
    agent_as_a_tool=pubchem_driver_based_agent,
    verbose=True
)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'MicrowaveSynthesizer', 'microservice_description': 'Module for a virtual microwave synthesizer.', 'additional_instructions': "\nIf you are unsure about a specific request or any information you observe use the 'human' tool to ask the human for guidance.\n\nDon't just use default values for tool inputs, look at the chat history to see if any values were provided previosuly and if not ask the human for input and provide the human with the default values if requested.\n"} template='You are an AI agent in control of the following microserivce - {microservice}. \n\nA short description of the microservice is as follows:\n\n{microservice_description}\n\nRespond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_in

In [33]:
initiator_and_pubchem_agent.invoke(
    {
        "input": "I am running a reaction with reactants 3-bromoquinoline and morpholine in vial 3. Please heat this vial at atmospheric pressure for 30 mins at 100 degrees C. Additionally, there are 5mg of each reactant, please include the number of mols of each reactant in your final answer using a trusted data source for the molecular weights as a reference."
    }
)

#? I do not think we have an endpoint that can get information about paramaters for a reaction and use that information to do the reaction.

#NOTE: we need a better question that also uses initiator+ along with pubchem

#TODO: will need to specify the reactants, products, and catalysts in the input
#Encourage it to use pubchem through the prompt and not its knowledge base - without a direct instruction



> Entering new AgentExecutor chain...
Question: I am running a reaction with reactants 3-bromoquinoline and morpholine in vial 3. Please heat this vial at atmospheric pressure for 30 mins at 100 degrees C. Additionally, there are 5mg of each reactant, please include the number of mols of each reactant in your final answer using a trusted data source for the molecular weights as a reference.

Thought: The user wants to run a reaction with specified reactants in vial 3. They provided the heating parameters as well. I need to set up a session, load the vial, update the heating parameters, and then heat the vial. Afterwards, I will need to calculate the number of moles of each reactant using the molecular weights of the reactants. However, I first need to start a session.

Action:
```
{
    "action": "allocate_session",
    "action_input": {}
}
```{'session_ID': 'a02ca2b1-9982-4696-9047-0f1f54b80523'}The session has been successfully initiated. Now, I need to load the vial into the micro

{'input': 'I am running a reaction with reactants 3-bromoquinoline and morpholine in vial 3. Please heat this vial at atmospheric pressure for 30 mins at 100 degrees C. Additionally, there are 5mg of each reactant, please include the number of mols of each reactant in your final answer using a trusted data source for the molecular weights as a reference.',
 'chat_history': '',
 'past_action_log': '',
 'output': 'The number of moles of 3-bromoquinoline is 0.005g / 208.05g/mol = 0.000024 moles. The number of moles of morpholine is 0.005g / 87.12g/mol = 0.000057 moles.',
 'intermediate_steps': [(AgentAction(tool='allocate_session', tool_input={}, log='Question: I am running a reaction with reactants 3-bromoquinoline and morpholine in vial 3. Please heat this vial at atmospheric pressure for 30 mins at 100 degrees C. Additionally, there are 5mg of each reactant, please include the number of mols of each reactant in your final answer using a trusted data source for the molecular weights as 

In [34]:
print(initiator_and_pubchem_agent.memory.memories[0].buffer)

Human: I am running a reaction with reactants 3-bromoquinoline and morpholine in vial 3. Please heat this vial at atmospheric pressure for 30 mins at 100 degrees C. Additionally, there are 5mg of each reactant, please include the number of mols of each reactant in your final answer using a trusted data source for the molecular weights as a reference.
AI: The number of moles of 3-bromoquinoline is 0.005g / 208.05g/mol = 0.000024 moles. The number of moles of morpholine is 0.005g / 87.12g/mol = 0.000057 moles.


In [35]:
print(initiator_and_pubchem_agent.memory.memories[1].buffer)

The human provides instructions for a chemical reaction involving reactants 3-bromoquinoline and morpholine in vial 3, to be heated at atmospheric pressure for 30 mins at 100 degrees C. The AI allocates a session, loads the vial, updates heating parameters, and begins heating the vial. The AI plans to calculate the number of moles of each reactant using their molecular weights.


# PubChem Agent + RAG

I would say that you are running the reaction list in the lab notebook provided by Aco with some ELN identifier for that specific notebook and ask it to give you the reactants and products

And then from there you can use PubChem on those reactants and products

Im not sure if Aco has an identifier on his ELNs but it should be at the top

How parameters change:

- ✅ rag_vectordb_path : `str` - Path to the VectorDB file.
- ✅ use_pubchem : `bool` - Use PubChem to get information whenever required.
- ❌ human_interaction: `bool` - Whether to use human interaction

In [ ]:
pubchem_and_rag_agent = create_sciborg_chat_agent(
    microservice=pubchem_command_microservice,
    rag_vectordb_path = str(project_root / 'embeddings' / 'NIH_Aco_only_doc_embeddings'),
    llm=ChatOpenAI(model='gpt-4'),
    human_interaction=False,
    verbose=True
)

# agent_as_a_tool=pubchem_driver_based_agent,
# agent_as_a_tool_description=pubchem_instructions,

input_variables=['agent_scratchpad', 'chat_history', 'input', 'past_action_log', 'tool_names', 'tools'] partial_variables={'microservice': 'PubChemCaller', 'microservice_description': 'This is the Module for Pubchem API caller.\n\nThis is one of the best Chemical databases. You have to use this database for any queries related to chemical structures, properties, biological activities, substance IDs (SIDs), compound IDs (CIDs), synonyms, descriptions, properties, assay details, or classification nodes.\n\nYou have access to specialized PubChem API functions for retrieving chemical and biological information. Whenever you encounter queries related to chemical structures, properties, biological activities, molecular weights, substance IDs (SIDs), compound IDs (CIDs), synonyms, descriptions, properties, assay details, or classification nodes, utilize the provided PubChem functions to fetch accurate and detailed information. Here are some functions you should use based on the type of inform

In [38]:
pubchem_and_rag_agent.invoke(
    {
        "input": "I am running a reaction listed in the lab notebook with ID: NCATS_RADUJEVICA2_0003_0003. Can you tell me what were the reactants and products used in this reaction?. "
    }
)



> Entering new AgentExecutor chain...
Thought: The user wants to know the reactants and products used in a specific reaction listed in the lab notebook with ID: NCATS_RADUJEVICA2_0003_0003. I don't have direct access to this information, so I will use the call_RAG_agent tool to query the relevant sources of information. 

Action:
```
{
    "action": "call_RAG_agent",
    "action_input": {"question": "What were the reactants and products used in the reaction listed in the lab notebook with ID: NCATS_RADUJEVICA2_0003_0003?"}
}
```

> Entering new AgentExecutor chain...

Invoking: `get_answer_from_information` with `{'situation': 'Reactants and products used in the reaction listed in the lab notebook with ID: NCATS_RADUJEVICA2_0003_0003'}`


[Document(page_content='Experiment\nNCATS_RADUJEVICA2_0003_0003\nMWAS Buchwald-Hartwig - rxn times screen\nNotebook\nNCATS_RADUJEVICA2_0003\nProject\nA2E\nCreated\nAco Radujevic (RADUJEVICA2)\naco.radujevic@nih.gov\n2024-03-28 21:06:16 GMT\nModified

{'input': 'I am running a reaction listed in the lab notebook with ID: NCATS_RADUJEVICA2_0003_0003. Can you tell me what were the reactants and products used in this reaction?. ',
 'chat_history': '',
 'past_action_log': '',
 'rag_log': '',
 'output': 'The reactants and products used in the reaction listed in the lab notebook with ID NCATS_RADUJEVICA2_0003_0003 are as follows:\n\nReactants:\n1. R1: 110-91-8 Amine (1.2 equivalents)\n2. R2: 5332-24-1 Aryl-Br (1.0 equivalent)\n3. R3: 1445085-82-4 PEPPSI (0.05 equivalent, formulation - ChemBeads)\n4. R4: 865-47-4 KOtBu (2.0 equivalents)\n\nSolvents:\n1. S1: 108-88-3 Toluene\n2. S1: 123-91-1 Dioxane\n\nProducts:\n1. P: 137794-83-3 ArylAmine\n\nThe reaction procedure involved adding the reactants in a specific order to a reaction vessel and carrying out the reaction under defined conditions.',
 'intermediate_steps': [(AgentAction(tool='call_RAG_agent', tool_input={'question': 'What were the reactants and products used in the reaction listed 

In [ ]:
#For followup questions
pubchem_and_rag_agent.invoke(
    {
        "input": "Can you give me the smiles representation of all the reactants and products?"
    }
)



> Entering new AgentExecutor chain...
Thought: The user is asking for the SMILES representation of all the reactants and products in the reaction. I can use the 'get_smiles_from_name' function to convert the names of the reactants and products into SMILES representations.
Action:
```
{
    "action": "get_smiles_from_name",
    "action_input": {
        "inp": "110-91-8 Amine, 5332-24-1 Aryl-Br, 1445085-82-4 PEPPSI, 865-47-4 KOtBu, 108-88-3 Toluene, 123-91-1 Dioxane, 137794-83-3 ArylAmine"
    }
}
```Error: 
 Staus code: 500
Message: <h1>Page not found (404)</h1>

 URL created: https://cactus.nci.nih.gov/chemical/structure/110-91-8 Amine, 5332-24-1 Aryl-Br, 1445085-82-4 PEPPSI, 865-47-4 KOtBu, 108-88-3 Toluene, 123-91-1 Dioxane, 137794-83-3 ArylAmine/smiles
NoneIt seems that the 'get_smiles_from_name' function did not return any output. This could be due to incorrect input formatting. The function expects a single chemical name as input, but I provided a list of names. I need to call 

{'input': 'Can you give me the smiles representation of all the reactants and products?',
 'chat_history': 'Human: I am running a reaction listed in the lab notebook with ID: NCATS_RADUJEVICA2_0003_0003. Can you tell me what were the reactants and products used in this reaction?. \nAI: The reactants and products used in the reaction listed in the lab notebook with ID NCATS_RADUJEVICA2_0003_0003 are as follows:\n\nReactants:\n1. R1: 110-91-8 Amine (1.2 equivalents)\n2. R2: 5332-24-1 Aryl-Br (1.0 equivalent)\n3. R3: 1445085-82-4 PEPPSI (0.05 equivalent, formulation - ChemBeads)\n4. R4: 865-47-4 KOtBu (2.0 equivalents)\n\nSolvents:\n1. S1: 108-88-3 Toluene\n2. S1: 123-91-1 Dioxane\n\nProducts:\n1. P: 137794-83-3 ArylAmine\n\nThe reaction procedure involved adding the reactants in a specific order to a reaction vessel and carrying out the reaction under defined conditions.',
 'past_action_log': 'The human asks the AI about the reactants and products used in a specific lab reaction listed i

In [40]:
print(pubchem_and_rag_agent.memory.memories[0].buffer)

Human: I am running a reaction listed in the lab notebook with ID: NCATS_RADUJEVICA2_0003_0003. Can you tell me what were the reactants and products used in this reaction?. 
AI: The reactants and products used in the reaction listed in the lab notebook with ID NCATS_RADUJEVICA2_0003_0003 are as follows:

Reactants:
1. R1: 110-91-8 Amine (1.2 equivalents)
2. R2: 5332-24-1 Aryl-Br (1.0 equivalent)
3. R3: 1445085-82-4 PEPPSI (0.05 equivalent, formulation - ChemBeads)
4. R4: 865-47-4 KOtBu (2.0 equivalents)

Solvents:
1. S1: 108-88-3 Toluene
2. S1: 123-91-1 Dioxane

Products:
1. P: 137794-83-3 ArylAmine

The reaction procedure involved adding the reactants in a specific order to a reaction vessel and carrying out the reaction under defined conditions.
Human: Can you give me the smiles representation of all the reactants and products?
AI: The SMILES representations for the provided chemical IDs are as follows:

- 110-91-8: C1COCCN1
- 5332-24-1: Brc1cnc2ccccc2c1
- 1445085-82-4: (Not availabl

In [41]:
print(pubchem_and_rag_agent.memory.memories[1].buffer)

The human asks the AI about the reactants and products used in a specific lab reaction listed in the notebook. The AI does not provide any information in response. The human then asks for the SMILES representation of all the reactants and products. The AI attempts to retrieve the SMILES representations for each chemical name provided, encountering some difficulties along the way.


In [42]:
print(pubchem_and_rag_agent.memory.memories[2].buffer)

The AI agent used the call_RAG_agent tool to query relevant sources of information and found the reactants, products, and solvents used in a specific reaction listed in the lab notebook with ID NCATS_RADUJEVICA2_0003_0003. The reactants and products used in the reaction are listed, along with the solvents. The AI agent referred to specific documents to gather this information for documentation and future reference.
